In [ ]:
!pip install beautifulsoup4
!pip install wikipedia
!pip install pandas

In [4]:
#Importing libraries
from bs4 import BeautifulSoup
import urllib
from urllib import request
import urllib.request as ur
import requests
import wikipedia
import random
from constants import *
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import random
all_titles = []


# Recursively get relevant wikipedia page links

In [5]:
# scrapes all the relevant titles.
def scrapeLinks(url, depth, num_links_per_page=10):

    # url filtering to remove wiki assets:
    if (len(url.split("/")) != 5):
        return 
    last_path = url.split("/")[4]
    if last_path.startswith("Wikipedia:") \
        or last_path.startswith("Category:")\
        or last_path.startswith("Template:") \
        or last_path.startswith("File:") \
        or last_path.startswith("Help:") \
        or last_path.startswith("Special:"):
        return

    global all_titles
    all_titles.append(url)
    
    # end condition
    if (depth == 0):
        return
    
    response = requests.get(
        url=url,
    )

    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find(id="firstHeading")
    allLinks = soup.find(id="bodyContent").find_all("a")
    random.shuffle(allLinks)

    # limit to top n links
    allLinks = allLinks[:num_links_per_page]
    
    # Recursively scrape links
    for link in allLinks:
        try: 
            if link['href'].find("/wiki/") == -1:
                continue
                
            # Use this link to scrape
            scrapeLinks("https://en.wikipedia.org" + link['href'], depth - 1)
        except:
            pass



# Preprocess each of the wikipedia pages and remove headers

In [ ]:
def scrapeWikiLinksToCSV(search_term):
    
    global all_titles
    
    # Search for similar wiki titles and try all relevant routes:
    relevant_routes = wikipedia.search(search_term)
    for i in relevant_routes:
        try:
            # construct url:
            start_url = wikipedia.page(i).url
            
            # scrapelinks and accumulate in global all_titles
            scrapeLinks(start_url, SEARCH_DEPTH, NUM_LINKS_PER_PAGE)
        except:
            pass
    
    df = pd.DataFrame(columns=["url", "title", "content"])
    wikipedia.set_lang("en")
    for link in all_titles:
        # get basepath:
        title = link.split('/')[-1]
        try:

            # only get the first NUM_SENTENCES_FROM_WIKI amount of sentences
            content = wikipedia.summary(title, sentences=NUM_SENTENCES_FROM_WIKI)
                
            # preprocess content to remove "==== xxx ===="
            content = re.sub(r'==.*?==', '', content)  
            
            # add to dataframe
            df.loc[len(df)] = [link, title, content]
        except:
            pass
    
    # save as csv.
    df.to_csv('scraped.csv', index=False)
    return df
    

# Run Below:

In [19]:
df = scrapeWikiLinksToCSV(INPUT_URL)

/opt/miniconda3/envs/chromerag/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/miniconda3/envs/chromerag/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/opt/miniconda3/envs/chromerag/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or 

In [20]:
df.head()

,url,title,content
0,https://en.wikipedia.org/wiki/Web_scraping,Web_scraping,"Web scraping, web harvesting, or web data extr..."
1,https://en.wikipedia.org/wiki/Document_Object_...,Document_Object_Model,The Document Object Model (DOM) is a cross-pla...
2,https://en.wikipedia.org/wiki/List_of_unit_tes...,List_of_unit_testing_frameworks#JavaScript,This is a list of notable test automation fram...
3,https://en.wikipedia.org/wiki/Blink_(browser_e...,Blink_(browser_engine),Blink is a browser engine developed as part of...
4,https://en.wikipedia.org/wiki/Serialization,Serialization,"In computing, serialization (or serialisation,..."
